In [1]:
import torch
from tqdm.auto import tqdm
from transformer_lens import HookedTransformer
from jaxtyping import Float, Int, Bool
from torch import Tensor
from tqdm.auto import tqdm
import plotly.io as pio
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import numpy as np
import plotly.express as px 
from collections import defaultdict
import matplotlib.pyplot as plt
import re
from IPython.display import display, HTML
from datasets import load_dataset
from collections import Counter
import pickle
import os
import haystack_utils
import hook_utils
from transformer_lens import utils
from fancy_einsum import einsum
import einops
import json
import ipywidgets as widgets
from IPython.display import display
from datasets import load_dataset
import random
import math
import random
import neel.utils as nutils
from neel_plotly import *
import csv

SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

pio.renderers.default = "notebook_connected+notebook"
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.autograd.set_grad_enabled(False)
torch.set_grad_enabled(False)

%reload_ext autoreload
%autoreload 2

In [13]:
model = HookedTransformer.from_pretrained("gpt2-large")

with open('data/gpt2_large_spectrum_pre_v2.pkl', 'rb') as f:
    sample_df = pickle.load(f)

with open('data/gpt2_large_spectrum_pre_bins_v2.pkl', 'rb') as f:
    bin_edges = pickle.load(f)

def read_labels(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'r') as f:
            return {row[0]: row[1] for row in csv.reader(f)}
    return {}

labels = read_labels('data/gp2_large_spectrum_labels_pre_v2.csv')
sorted_index_labels = sorted([(int(index), int(label)) for index, label in labels.items()])
sorted_labels = np.array([label for index, label in sorted_index_labels])

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-large into HookedTransformer


In [14]:
sample_df['label'] = sorted_labels

In [4]:
text = load_dataset("stas/openwebtext-10k", split="train")

Repo card metadata block was not found. Setting CardData to empty.


In [ ]:
# high_an_samples = sample_df[sample_df["L31N892 Activation"] == pd.Interval(6.473, 6.823, 'right')]

# for i in range(10):
#     sample = high_an_samples.iloc[i]
#     print(model.to_str_tokens(model.to_tokens(text['text'][sample['prompt_index']])[0, :sample['token_index']+2]))

# sample_df[sample_df["L31N892 Activation"] == pd.Interval(6.473, 6.823, 'right')]

In [15]:
grouped_total = sample_df.groupby('L31N892 Activation', as_index=False).count()['label']
grouped_3 = sample_df[sample_df['label'] == 3].groupby('L31N892 Activation', as_index=False).count()['label'] / grouped_total
grouped_3 = grouped_3.fillna(0)
grouped_2 = sample_df[sample_df['label'] == 2].groupby('L31N892 Activation', as_index=False).count()['label'] / grouped_total
grouped_2 = grouped_2.fillna(0)
grouped_1 = sample_df[sample_df['label'] == 1].groupby('L31N892 Activation', as_index=False).count()['label'] / grouped_total
grouped_1 = grouped_1.fillna(0)

df = pd.DataFrame({
    'L31N892 Activation': grouped_3.index,
    'Next token is \' an\'' : grouped_3.values,
    'Next token is plausible \' an\'': grouped_2.values,
    'Next token is not \' an\'': grouped_1.values
})

# Cut off tails, 20 data points per


In [16]:
grouped_melted = df.melt(id_vars=['L31N892 Activation'], value_vars=['Next token is \' an\'', 'Next token is plausible \' an\'', 'Next token is not \' an\''], 
                            var_name='status', value_name='Proportion of Labels')

In [17]:
# grouped = sample_df.groupby('L31N892 Activation', as_index=False)

# grouped['Next token is \' an\''] = grouped_3

# grouped['Next token is plausible \' an\''] = grouped_2
# grouped['Next token is not \' an\''] = grouped_1

fig = px.area(grouped_melted, x='L31N892 Activation', y='Proportion of Labels', color='status', line_shape='spline')
tickvals = np.arange(len(bin_edges) - 1)
ticktext = [f"{bin_edges[i]:.2f}-{bin_edges[i+1]:.2f}" for i in tickvals]

fig.update_layout(
    yaxis=dict(range=[0, 1]),
    xaxis=dict(
        tickmode='array',
        tickvals=tickvals,
        ticktext=ticktext
    ),
)
fig.show()

In [12]:
grouped.head()

NameError: name 'grouped' is not defined